<a href="https://colab.research.google.com/github/yasirabd/solver-society-job-data/blob/main/2_3_1_Ekstrak_job_description_(pendidikan).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objective
Ekstrak job position, hasil ekstraksi adalah:
1. **Pendidikan**: kualifikasi pendidikan

Data input yang dibutuhkan:
1. jobstreet_clean_tahap2.csv

Data output yang dihasilkan:
1. jobstreet_clean_tahap3.csv

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [2]:
jobstreet = pd.read_csv("/content/drive/My Drive/Data Loker/jobstreet_clean_tahap2.csv")

# check null
jobstreet['job_description'].isnull().sum()

0

In [3]:
job_description = jobstreet['job_description'].str.lower()

job_description.head()

0    [sep][sep]usia max. 35 tahun[sep]pendidikan mi...
1    responsibilities:[sep]brief description of dut...
2    [sep]lingkup pekerjaan: [sep]menjaga account d...
3    [sep]candidate must possess at least diploma i...
4    [sep]lingkup pekerjaan: [sep]menjaga account d...
Name: job_description, dtype: object

## pendidikan

In [4]:
# cari job description yang mengandung 
# kata kunci 'pendidikan', 'degree', 'lulusan'
temp = list(job_description.values)

count = 0
for idx, job_desc in enumerate(temp):
    if 'pendidikan' in job_desc:
        count += 1
    elif 'degree' in job_desc:
        count += 1
    elif 'lulusan' in job_desc:
        count += 1
count

9636

In [5]:
list_pendidikan = ['smp', 'sma', 'smk', 'slta', 'smu', 'stm',
                   'diploma', 'd3', 'diii', 'd-iii', 'd.3', 'd4',
                   'sarjana','s1', 's-1', 'strata-1', 's2', 'strata-2' 
                   's-2', 's3', 's-3', 'strata-3',
                   'bachelor', 'diploma', 'master', 'post', 'graduate', 'professional']
pendidikan_mapper = {'smp':'smp',
                     'slta':'sma', 
                     'smu':'sma',
                     'stm':'smk',
                     'diploma':'diploma',
                     'd3':'d3',
                     'diii':'d3',
                     'd-iii':'d3',
                     'd.3':'d3',
                     'd4':'d4',
                     'sarjana':'sarjana',
                     's1':'s1',
                     's-1':'s1',
                     'strata-1':'s1',
                     's2':'s2',
                     'strata-2':'s2',
                     's-2':'s2',
                     's3':'s3',
                     's-3':'s3',
                     'strata-3':'s3',
                     'bachelor':'s1',
                     'master':'s2',
                     'postgraduate':'s3',
                     'professional':'profesi'}
                     

extract_pendidikan = list()

for job_desc in list(job_description.values):
    # replace token [sep]
    job_desc = job_desc.replace('[sep]', ' ')
    job_desc = job_desc.replace(':', ' ')

    # split dengan space
    split_desc = job_desc.split()

    item_match = list()
    for item in split_desc:
        if any(x in item for x in list_pendidikan):
            # cleaning punctuation
            item = re.sub(r"[(,.;@#?!&$)/]+\ *", " ", item).strip()
            
            # split from 's
            item = item.split("'")[0]
            item = item.split("’")[0]

            # split
            for it in item.split():
                item_match.append(it)
    
    # join 'post' dan 'graduate'
    if 'post' in item_match and 'graduate' in item_match:
        item_match.remove('post')
        item_match.remove('graduate')
        item_match.append('postgraduate')

    # mapping pendidikan
    item_match = pd.Series(item_match).map(pendidikan_mapper).dropna().drop_duplicates().tolist()

    if len(item_match) >=1:
        extract_pendidikan.append(item_match)
    else:
        extract_pendidikan.append(['tidak dicantumkan'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [6]:
# lihat hasil ekstraksi
extract_pendidikan[:5]

[['d3'], ['s1'], ['d3', 's1', 'sma'], ['diploma'], ['d3', 's1', 'sma']]

In [7]:
# lihat banyak loker yang tidak mencantumkan pendidikan
count = 0
for p in extract_pendidikan:
    if 'tidak dicantumkan' in p:
        count +=1
count

2870

In [8]:
# check unique pendidikan
temp = list()
for p in extract_pendidikan:
    temp += p

pd.Series(temp).value_counts()

s1                   7414
tidak dicantumkan    2870
diploma              2115
d3                   1993
sarjana              1184
profesi               629
s2                    629
sma                   499
s3                    326
smk                    54
d4                     20
smp                     9
dtype: int64

In [9]:
# lakukan mapping lagi
pend_mapper = {'s1': 's1',
               'tidak dicantumkan': 'tidak dicantumkan',
               'diploma': 'd3',
               'd3': 'd3',
               'sarjana': 's1',
               'profesi': 'profesi',
               's2': 's2',
               'sma': 'sma',
               's3': 's3',
               'smk': 'smk',
               'd4': 'd4',
               'smp': 'smp'}

In [10]:
result = list()
for p in extract_pendidikan:
    mapping = pd.Series(p).map(pend_mapper).drop_duplicates()
    join = ', '.join(list(mapping)).strip()

    result.append(join)

In [11]:
# check unique pendidikan lagi
temp = list()
for p in result:
    split = [data.strip() for data in p.split(',')]
    temp += split

pd.Series(temp).value_counts()

s1                   8491
d3                   4010
tidak dicantumkan    2870
profesi               629
s2                    629
sma                   499
s3                    326
smk                    54
d4                     20
smp                     9
dtype: int64

In [12]:
jobstreet['pendidikan'] = result
jobstreet['pendidikan'].isnull().sum()

0

In [13]:
jobstreet.head()

,id_loker,pulau,provinsi,job_position,company_name,years_of_experience,years_of_experience_cat,job_description,average_processing_time,company_industry,company_size,work_environment_waktu_bekerja,work_environment_gaya_berpakaian,work_environment_gaya_berpakaian_contoh,work_environment_tunjangan,work_environment_bahasa,platform,pendidikan
0,1,jawa,dki jakarta,spv finance,PT Axindo Infotama,3,0 - 4 tahun,[SEP][SEP]Usia Max. 35 tahun[SEP]Pendidikan Mi...,6,Retail/Merchandise,51 - 200 pekerja,sabtu,formal,kemeja polos & dasi,"kesehatan, umum",indonesia,jobstreet,d3
1,2,jawa,dki jakarta,qualitative research executive,Perusahaan Dirahasiakan,3,0 - 4 tahun,RESPONSIBILITIES:[SEP]Brief Description of Dut...,tidak tersedia,tidak tersedia,tidak spesifik,tidak tersedia,tidak spesifik,tidak spesifik,tidak tersedia,tidak dicantumkan,jobstreet,s1
2,3,jawa,jawa tengah,field collection wonogiri,OTO Group,0,0 - 4 tahun,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,senin - jumat,bisnis,kemeja,"kesehatan, bonus",indonesia,jobstreet,"d3, s1, sma"
3,4,jawa,dki jakarta,perawat di negara jerman,PT Jerman Manajemen,1,0 - 4 tahun,[SEP]Candidate must possess at least Diploma i...,11,Pendidikan,51 - 200 pekerja,sabtu - kamis,formal,kemeja polos & dasi,kesehatan,indonesia,jobstreet,d3
4,5,jawa,jawa tengah,field collection pati,OTO Group,0,0 - 4 tahun,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,senin - jumat,bisnis,kemeja,"kesehatan, bonus",indonesia,jobstreet,"d3, s1, sma"


In [14]:
jobstreet.isnull().sum()

id_loker                                   0
pulau                                      0
provinsi                                   0
job_position                               0
company_name                               0
years_of_experience                        0
years_of_experience_cat                    0
job_description                            0
average_processing_time                    0
company_industry                           0
company_size                               0
work_environment_waktu_bekerja             0
work_environment_gaya_berpakaian           0
work_environment_gaya_berpakaian_contoh    0
work_environment_tunjangan                 0
work_environment_bahasa                    0
platform                                   0
pendidikan                                 0
dtype: int64

# Export csv

In [15]:
jobstreet.to_csv('jobstreet_clean_tahap3.csv', index=False)